# pySEAFOM Crosstalk Test — Synthetic Data

This notebook demonstrates the **crosstalk** calculation and plotting workflow using the `pySEAFOM.crosstalk` module.

## What this notebook does
- Generates synthetic DAS section data centered on a stimulation point
- Extracts the stimulus-frequency magnitude per SSL via block FFT
- Computes a crosstalk profile (dB re reference) and a max crosstalk metric
- Repeats the same single-section calculation for multiple sections (looping)

## 1) Setup and Imports

Adds the local `source` directory to `sys.path` so you can run this notebook without installing the package.

In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

repo_root = Path.cwd().parent
src_dir = repo_root / 'source' / 'pySEAFOM'
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

import importlib
import crosstalk  # if using relative import
importlib.reload(crosstalk)

#from pySEAFOM import crosstalk # if using installed package import

## 2) Configure Test Parameters

This mirrors the call pattern you provided: `compute_crosstalk(section_data=..., stimulus_freq=..., fs=..., fft_size=..., gauge_length=..., stretcher_length=..., channel_spacing=...)`.

In [ ]:
fs = 10000
stimulus_freq = 500
gauge_length = 2.0
channel_spacing = 1.0  
test_sections = ['Section 1', 'Section 2']

strecher_sections_channels=[[25,35],[65,75]]
gl_multiplier=5


## 3) Create Synthetic Data [Optional - Skip if Using Real Data]

We synthesize a sinusoid at `stimulus_freq` whose amplitude is strongest at the center SSL (stimulation point) and decays with distance.

We generate multiple spatial sections in a full `(channels × samples)` array, then slice out each test section and run the single-section crosstalk computation in a loop.

In [ ]:
duration_s = 60
stretcher_length = 10.0
n_channels = 100
n_samples = int(fs * duration_s)
stretcher_nch = int(stretcher_length / channel_spacing)
rng = np.random.default_rng(123)
t = np.arange(n_samples) / fs

def make_section(start_ch, end_ch, noise_sigma=0.03, base_amp=1.0):
    n_ssl = end_ch - start_ch + 1
    section = np.empty((n_ssl, n_samples), dtype=np.float32)

    # compute stretcher zone inside this section
    mid = n_ssl // 2
    half = stretcher_nch // 2
    stretch_start = max(0, mid - half)
    stretch_end   = min(n_ssl - 1, mid + half)

    # amplitude envelope: high inside stretcher, low outside
    amp = np.ones(n_ssl, dtype=float) * 0.1
    amp[stretch_start:stretch_end+1] = base_amp

    for i in range(n_ssl):
        phase = float(2 * np.pi * rng.random())
        sig = amp[i] * np.sin(2 * np.pi * stimulus_freq * t + phase)
        sig = sig + noise_sigma * rng.standard_normal(n_samples)
        section[i, :] = sig.astype(np.float32)

    return section
# build all sections dynamically
sections = []
for (start_ch, end_ch) in strecher_sections_channels:
    sec = make_section(start_ch, end_ch)
    sections.append(sec)

# allocate full array
data = np.zeros((n_channels, n_samples), dtype=np.float32)
# --- background noise for all channels ---
background_sigma = 0.2   # adjust as needed
data = background_sigma * rng.standard_normal((n_channels, n_samples)).astype(np.float32)
# fill it with each section
for (start_ch, end_ch), sec in zip(strecher_sections_channels, sections):
    row_start = start_ch 
    row_end   = row_start + sec.shape[0]
    data[row_start:row_end, :] = sec

plt.figure(figsize=(12, 4))
plt.imshow(data[:, :2000], aspect='auto', cmap='seismic')
plt.colorbar()
plt.title("Full Data Array (first 2000 samples)")
plt.xlabel("Samples")
plt.ylabel("Channel index")
plt.show()


## 4) Compute Crosstalk (per section)

This returns a per-SSL crosstalk profile (dB re reference) and a `max_xt_db` metric for each section.

In [ ]:
for i in range(len(test_sections)):
    st_index = int(max(0, strecher_sections_channels[i][0] - gl_multiplier * gauge_length / channel_spacing))
    end_index = int(min(n_channels - 1, strecher_sections_channels[i][1] + gl_multiplier * gauge_length / channel_spacing))

    # Slice channels, keep all time samples: (n_ssl, n_samples)
    test_section = data[st_index:end_index + 1, :]

    result = crosstalk.calculate_crosstalk(
        section_data=test_section,
        stimulus_freq=stimulus_freq,
        fs=fs,
        gauge_length=gauge_length,
        stretcher_length=stretcher_length,
        channel_spacing=channel_spacing,
        gl_multiplier=gl_multiplier,
    )

    print(f"{test_sections[i]}: max_xt = {result['max_xt_db']:.2f} dB | reference_level = {result['reference_level']:.3e}")

    title = f"Crosstalk (Synthetic) — GL={gauge_length} m — {test_sections[i]}"
    fig, ax = crosstalk.plot_crosstalk(
        result["crosstalk_db"],
        channel_spacing=channel_spacing,
        title=title,
        label=test_sections[i],
    )
    plt.show()